In [224]:
#導入相關模組
import json
import random
import re

import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#CKIPtagger導入要用的模組，分別有「WS（斷詞）」、「POS（詞性標注）」及「NER（實體辨識）」與「construct__dictionary(自建辭典)」四個功能。
from ckiptagger import construct_dictionary, WS, POS, NER
ws = WS('E:/Python 3.7/pyetl/Demodb0103/gym/data')
pos = POS('E:/Python 3.7/pyetl/Demodb0103/gym/data')
ner = NER('E:/Python 3.7/pyetl/Demodb0103/gym/data')

#讀取要段詞的json檔
DATASET_DIR = 'E:/Python 3.7/pyetl/Demodb0103/gym/data/gymz.json'
with open(DATASET_DIR, encoding='utf8') as f:
    dataset = json.load(f)
    
import time   

In [225]:
#讀取停止詞典
STOP_WORDS_DIR = 'E:/Python 3.7/pyetl/Demodb0103/gym/data/stopWords.txt'
with open(STOP_WORDS_DIR, encoding='utf8') as f:
    stop_words = f.read().splitlines() 

In [226]:
#抽出文章的標題與內容
content_list = list(map(lambda d: d['content'], dataset))
title_list = list(map(lambda d: d['title'], dataset))

In [227]:
#設自建辭典，並給予權重
word_to_weight1 = {
    "胸肌": 3,
    "胸部": 3,
    "肩膀": 3,
    "背肌": 3,
    "背部": 3,
    "手臂": 3,
    "腹肌": 3,
    "腹部": 3,
    "核心": 3,
    "腿部": 3,
    "深蹲": 3,
    "硬舉": 3,
    "臥推": 3,
    "啞鈴": 3,
    "有氧運動": 3,
    "徒手訓練": 3,
    "徒手": 3,
    "肌肉": 3,
    "重訓": 3,
    "健身": 3,
    "弓箭步": 3,
    "伏地挺身": 3,
    "引體向上": 3,
    "棒式": 3,
    "仰臥起坐": 3,
    "捲腹": 3,
    "跑步": 3,
    "開合跳": 3,  
    "橋式": 3,
    "增肌": 2,
    "減脂": 2,
}
dictionary1 = construct_dictionary(word_to_weight1)

word_to_weight2 = {
    "胸": 3,
    "上胸": 2,
    "下胸": 2,
    "肩": 3,
    "手": 3,
    "腿": 3,
    "斜方肌": 2,
    "二頭肌": 2,
    "三頭肌": 2,
    "小腿肌群": 2,
    "臀大肌": 2,
    "大腿肌": 2,
    "大腿後肌": 2,
    "胸大肌": 2,
    "前三角": 2,
    "肱三頭": 2,
    "中三角": 2,
    "三角肌": 2,
    "後三角": 2,
    "棘上肌": 2,
    "小圓肌": 2,
    "棘下肌": 2,
    "中斜方": 2,
    "菱形肌": 2,
    "肱二頭": 2,
    "背闊肌": 2,
    "豎棘肌": 2,
    "股四頭": 2,
    "腿後建肌群": 2,
    "核心肌群": 3,
    "腿內收肌群": 2,
    "臀中肌": 2,
    "緋腸肌": 2,
    "比目魚肌": 2,
    "腹直肌": 2,
    "腹橫肌": 2,
    "單邊弓箭步": 3,
    "上斜伏地挺身": 3,
    "下斜伏地挺身": 3,
    "暴力上槓": 3,
    "棒式撐體": 3,
    "反向捲腹": 3,
    "槓鈴": 2,
    "繩索": 1,
    "戰神": 1,
    "沙袋": 1,
    "按摩池": 1,
    "機械": 1,
    "跑步機": 1,
    "踏步機": 1,
    "飛輪腳踏車": 1,
    "交叉滑步訓練機": 1,
    "坐姿推胸機": 1,
    "肩部推舉機": 1,
    "坐姿划船機": 1,
    "旋轉訓練機": 1,
    "臂部複合訓練機": 1,
    "腿部外彎機": 1,
    "腿部內彎機": 1,
    "垂直蹬腿練習器": 1,
    "羅馬凳": 1,
    "單杠提膝器": 1,
    "背肌訓練機": 1,
    "心肺適能訓練": 1,
    "上斜跑步機": 1,
    " 跳繩": 1,
    "胸推": 1,
    "館長": 1,
    "胸飛鳥": 1,
    "上斜胸推": 1,
    "上斜飛鳥": 1,
    "下斜胸推": 1,
    "下斜飛鳥": 1,
    "肩推": 1,
    "前平舉": 1,
    "側平舉": 1,
    "二頭彎舉": 1,
    "反向飛鳥": 1,
    "內旋": 1,
    "外旋": 1,
    "滑輪下拉": 1,
    "反手": 1,
    "寬握": 1,
    "正常握": 1,
    "窄握": 1,
    "高划船": 1,
    "仰臥臂屈伸": 1,
    "俯身划船": 1,
    "單臂划船": 1,
    "單邊深蹲": 2,
    "腿推": 1,
    "腿伸": 1,
    "腿曲": 1,
    "腿內收訓練": 1,
    "腿外展訓練": 1,
    "臀部伸展訓練": 1,
    "舉重": 1,
    "組數": 1,
    "次數": 1,
    "節奏": 1,
    "休息": 1,
    "健身營養補給品": 1,
    "乳清蛋白": 1,
    "蛋白質營養棒": 1,
    "支鏈氨基酸": 1,
    "蛋白質": 1,
    "碳水化合物": 1,
    "鈉": 1,
    "脂肪": 1,
    "結實的": 2,
    "身材健美的": 2,
    "健康": 1,
    "精實的": 1,
    "肌肉拉傷": 2,
    "健身狂人": 2,
    "健身房會員": 1,
    "隱形肥胖": 2,
    "肥胖": 2,
    "脂肪過量": 2,
    "低體重": 2,
    "標準體型": 2,
    "肌肉型": 2,
    "運動員型": 2,
    "運動員": 2,
    "低脂肪": 2,
    "肘伸展": 1,
    "生酮": 1,
}
dictionary2 = construct_dictionary(word_to_weight2)


In [228]:
#替換不需要的句子，並正規化
# start = time.clock()
gym_list =[]
for i in content_list:
    i = re.sub('※ 發信站: 批踢踢實業坊',' ', i)
    i = re.sub('※ 文章網址:',' ', i)
    i = re.sub('※ 編輯:',' ', i)
    i = re.sub('(臺灣)',' ', i)
    i = re.sub('來自:',' ', i)
    #只保留中文
    rule = re.compile(r"[^\u4e00-\u9fa5]")
    i = rule.sub('', i)
    gym_list.append(i)


In [229]:
#進行ckip中研院斷詞
word_s = ws(gym_list,
            #考慮分隔符
            sentence_segmentation=True,
            #recommend_dictionary參考詞典
            recommend_dictionary = dictionary2,
            #coerce_dictionary強制詞典
            coerce_dictionary = dictionary1,
            #斷詞參考符號
            segment_delimiter_set={'?', '？', '!', '！', '。', ',',   
                                   '，', ';', ':', '、'})


In [230]:
#去除停止詞
for idx, speech in enumerate(word_s):
    word_s[idx] = ' '.join([word for word in speech if word not in stop_words])
# end = time.clock()
# print('運行時間: ' + str(end - start))

In [284]:
#Word2Vec
# 資料形式為一個文檔各為list元素為字串
from gensim.models import Word2Vec

#全文章轉為各list包起來
doc_clean1 = [doc.split() for doc in word_s]

# 建立模型
#window:CBOW下決定Word2Vec一次取多少詞來預測中間詞
#min_count:出現次數大於等於min_count的詞，才會納入Word2Vec的詞典中
#negative:Negative Sampling的取樣數量，5~20適合小數據，2~5適合大數據
# worker=使用多核計算機進行更快的訓練
model = Word2Vec(doc_clean1,window=5, negative=5,min_count=1)

# # 基於2d PCA擬合數據
# X = model[model.wv.vocab]
# pca = PCA(n_components=2)
# result = pca.fit_transform(X)

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [277]:
#顯示指定詞的關聯詞
model2.wv.most_similar('肥')

[('瘦子', 0.9213787913322449),
 ('胖子', 0.9148010015487671),
 ('瘦胖子', 0.9103908538818359),
 ('腿頸', 0.9088810086250305),
 ('瘦子練壯', 0.9006045460700989),
 ('死胖子', 0.890883207321167),
 ('胖', 0.8893124461174011),
 ('泡芙出', 0.8852933645248413),
 ('泡芙', 0.8737491369247437),
 ('肥宅臉', 0.872066855430603)]

In [271]:
#顯示各個指定詞的關聯詞
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df
most_similar(model, ['增肌', '肥'])

,增肌,cos,肥,cos
0,減脂,0.931740,瘦子,0.943572
1,肪,0.907604,胖子,0.932490
2,期,0.878710,胖,0.898200
3,增脂,0.819122,瘦下來,0.894688
4,增到,0.818283,肉肉,0.880289
5,壯起來,0.804180,變腫,0.873781
6,增重,0.795238,紙片人,0.871549
7,爬增,0.787592,瘦瘦,0.869552
8,拖垮,0.778630,身形,0.867059
9,小琉球,0.777449,復,0.861484


In [265]:
# import word2vec #需要word2vec模組
# indexes = model.cosine(u'增肌')
# for index in indexes[0]:
#     print (model.vocab[index])

ModuleNotFoundError: No module named 'word2vec'

In [ ]:
# fasttext
from gensim.models import FastText
model2 = FastText(doc_clean1,window=5, negative=5,min_count=1)
